# Usage embedder and evaluator

In [1]:
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from eden.graph import vectorize
import numpy as np
import umap

import load_data as ld
from embedder import embedder
from evaluator import evaluator

Load the dataset and split graphs and labels

In [9]:
name="PROTEINS"
graphs_label = ld.load_data(name)
#load the data
graphs_label = ld.load_data(name)
graphs_label = np.array(graphs_label)

# get graphs and labels
graphs = graphs_label[:,0]
labels = graphs_label[:,1]

Split the graphs in test and train

In [10]:
# split the graphs and the labels
X_train, X_test, y_train, y_test = train_test_split(graphs, labels, test_size=0.1, random_state=0)
y_test = [[i] for i in y_test]
y_test = np.ravel(y_test)
y_train = [[i] for i in y_train]
y_train = np.ravel(y_train)

create an array of dimensions, it rappresent the size of the dimension at the end of the embedding

In [11]:
mean_accuracy = []
std_accuracy = []
dim = np.logspace(0, 5, num=6, base=2)
for d in dim:
    # dimension of the last embedding
    dimension_embedding = int(d)

    #instanziate models
    pca = TruncatedSVD(n_components=dimension_embedding)
    vectoriz = "vectorize"
    
    embed = embedder(pca,vectoriz)
    evalu = evaluator()
    
    embed.fit(X_train)
    Xd_train = embed.transform(X_train)
    
    evalu.fit(Xd_train,y_train)
    
    
    Xd_test = embed.transform(X_test)
    mean,std = evalu.evaluate_with_kfold(Xd_test, y_test, 6)

    mean_accuracy.append(mean)
    std_accuracy.append(std)

mean_accuracy = np.array(mean_accuracy)
std_accuracy = np.array(std_accuracy)


KeyboardInterrupt: 

Plot mean and standard deviation of k fold corss validation when the size increases

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cbook as cbook

def plot(mean_accuracy,dim,std_accuracy):
    # the 1 sigma upper and lower analytic population bounds
    lower_bound = mean_accuracy - std_accuracy
    upper_bound = mean_accuracy + std_accuracy

    fig, ax = plt.subplots(1)
    ax.semilogx(dim, mean_accuracy, lw=2, label='accuracy', color='blue')
    ax.scatter(dim, mean_accuracy, color='blue')
    ax.fill_between(dim, lower_bound, upper_bound, facecolor='yellow', alpha=0.5,
                    label='std')
    ax.legend(loc='lower right')

    m = min(mean_accuracy)-max(std_accuracy)

    ax.set_xlabel('num steps')
    ax.set_ylabel('position')
    plt.ylim([m, 1.0])
    ax.grid()

    
plot(mean_accuracy,dim,std_accuracy)

2.0